In [2]:
"""
Code to check if Resilience & Reliability are Correctly Calculated
Tomás Schmieder & Maoz Bizan
CEE 4200
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


file_path = '/Users/windy/desktop/data.csv'
df = pd.read_csv(file_path)

# load the dataset, which has historical values for everything
# you will need to replace the file path
file_path = '/Users/windy/desktop/data.csv'
df = pd.read_csv(file_path)

# constants
s_c = 337                       # critical storage, 20% of max
D = 2.8512                      # demand of SPMR, mcm/day
max_storage = 1685              # max storage mcm
initial_storage = 1023.8908     # intial storage mcm


df["Change in time (days)"] = df["Change in time (days)"].fillna(method='ffill')
df["Combined Precipitation (mcm)"] = df["Combined Precipitation (mcm)"].fillna(0)
df["Inverted Inflow (mcm)"] = df["Inverted Inflow (mcm)"].fillna(0)


def percentfull(storage, maximum_storage):
    return (storage / maximum_storage) * 100

def revised_outflow_policy(storage_percentage):
    """
    Apply the specified outflow policy.
    """
    if storage_percentage >= 82.1:
        return 4.965
    elif 38.2 <= storage_percentage < 82.1:
        return 0.053
    elif 31.6 <= storage_percentage < 31.6:
        return 2.961
    elif 10.0 <= storage_percentage < 33.0:
        return 2.678
    else:
        return 1.893

def reliability(u, D):
    """
    Calculate reliability.
    """
    return u / D

def resilience(s_t, s_c):
    """
    Calculate resilience.
    """
    return (s_t - s_c) / s_c if s_c != 0 else float('nan')  # DONT REMOVE 


def simulate_storage():
    current_storage = initial_storage
    reliability_values = []
    resilience_values = []

    for i in range(148):
        precip = df["Combined Precipitation (mcm)"].iloc[i]
        inflow = df["Inverted Inflow (mcm)"].iloc[i]
        change_time = df["Change in time (days)"].iloc[i]
        storage_percentage = percentfull(current_storage, max_storage)
        outflow = revised_outflow_policy(storage_percentage)
        outflow_mcm = outflow * change_time

        next_storage = current_storage + precip + inflow - outflow_mcm
        next_storage = max(0, min(next_storage, max_storage))

        # append
        reliability_values.append(reliability(outflow, D))
        resilience_values.append(resilience(current_storage, s_c))

        # update
        current_storage = next_storage
    # average
    avg_reliability = np.mean(reliability_values)
    avg_resilience = np.mean(resilience_values)

    return avg_reliability, avg_resilience

average_reliability, average_resilience = simulate_storage()

# print
print("Average Reliability:", average_reliability)
print("Average Resilience:", average_resilience)


Average Reliability: 0.9381825006825008
Average Resilience: 3.1292649184008727
